In [ ]:
%load_ext autoreload
%autoreload
! pip install openai
! pip install tiktoken


In [34]:
import openai
import pandas as pd
import numpy as np
import json
import os

In [35]:
import tiktoken

openai.api_key  ='sk-'

In [37]:
df= pd.DataFrame(pd.read_csv("/content/test_text.txt", sep="\t", header= None))
df=df.rename(columns={0: "Text"})
df.head()

,Text
0,@user @user what do these '1/2 naked pics' hav...
1,OH: “I had a blue penis while I was this” [pla...
2,"@user @user That's coming, but I think the vic..."
3,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast..."


In [38]:
df

,Text
0,@user @user what do these '1/2 naked pics' hav...
1,OH: “I had a blue penis while I was this” [pla...
2,"@user @user That's coming, but I think the vic..."
3,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast..."
...,...
12027,Sentinel Editorial: FBI’s Comey ‘had no one of...
12028,perfect pussy clips #vanessa hudgens zac efron...
12029,#latestnews 4 #newmexico #politics + #nativeam...
12030,Trying to have a conversation with my dad abou...


In [41]:
df.Text.isnull().any()

False

In [39]:
df = df.dropna()
df = df[df['Text'].str.strip() != '']
df

,Text
0,@user @user what do these '1/2 naked pics' hav...
1,OH: “I had a blue penis while I was this” [pla...
2,"@user @user That's coming, but I think the vic..."
3,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast..."
...,...
12027,Sentinel Editorial: FBI’s Comey ‘had no one of...
12028,perfect pussy clips #vanessa hudgens zac efron...
12029,#latestnews 4 #newmexico #politics + #nativeam...
12030,Trying to have a conversation with my dad abou...


In [21]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['Text'] = df['Text'].apply(remove_punctuation)

In [32]:
for i,e in df.iterrows():
  if not e.Text:
    print(i)

In [22]:
df['Text']

0        user user what do these 12 naked pics have to ...
1        OH “I had a blue penis while I was this” playi...
2        user user Thats coming but I think the victims...
3        I think I may be finally in with the in crowd ...
4        user Wowfirst Hugo Chavez and now Fidel Castro...
                               ...                        
12027    Sentinel Editorial FBI’s Comey ‘had no one of ...
12028    perfect pussy clips vanessa hudgens zac efron ...
12029    latestnews 4 newmexico politics  nativeamerica...
12030    Trying to have a conversation with my dad abou...
12031    user You are a stand up guy and a Gentleman Vi...
Name: Text, Length: 12032, dtype: object

In [45]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

In [42]:
def sentiment(df, text_column):
    delimiter = "####"
    system_message = f"""As a social scientist, Your task is to analyze the sentiment of {delimiter} of user tweets extracted from Twitter. Please assign a sentiment score from 0 to 2 for each tweet, where 0 signifies negative sentiment, 1
indicates neutral sentiment, and 2 corresponds to positive sentiment. In situations where the sentiment is difficult to definitively classify, please provide your best estimation
of the sentiment score just return int numbers."""

    system_message = {'role': 'system', 'content': system_message}

    # Prepare an empty list to store the processed rows
    processed_rows = []
    i = 0
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the text from the specified 'text_column'
        text = row[text_column]

        # Prepare user messages for the current row
        user_messages = [{'role': 'user', 'content': f"{delimiter}{text}{delimiter}"}]

        # Create a list of messages for the current row, starting with the system message
        messages = [system_message] + user_messages

        # Get the model response for the current row
        response = get_completion_from_messages(messages).strip()
        processed_rows.append({**row, 'response': response})
    processed_df = pd.DataFrame(processed_rows)

    # Assuming you want to rename the 'text_column' to 'Text' in the final DataFrame
    processed_df.rename(columns={text_column: 'Text'}, inplace=True)

    return processed_df


In [ ]:
results_df=(sentiment(df.head(300),'Text'))

In [ ]:
results_df.response

0     0
1     0
2     0
3     2
4     1
     ..
95    0
96    2
97    0
98    0
99    2
Name: response, Length: 100, dtype: object

In [ ]:
df_label=pd.DataFrame(pd.read_csv("/content/test_labels.txt", sep="\t", header=None))
df_label.rename(columns={0:"Label"}, inplace= True)